In [68]:
import pandas as pd
import numpy as np
import datetime
import pytz

In [69]:
from google.colab import drive
drive.mount('/content/drive')

debug = False 

Mounted at /content/drive


In [70]:
# Modify the name of the picklist file as per actuals
folder_path = '/content/drive/MyDrive/stock'
transfer_file = f'{folder_path}/the-new-picklist.xlsx'

# modify result file, if required
current_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')) 
the_date = current_time.strftime("%Y-%m-%d-%H-%M-%S")

stock_file = f"{folder_path}/{the_date}-Stock-Status.xlsx"
debug_file = f"{folder_path}/{the_date}-Debug.xlsx"

In [71]:
opening_file_sheet = "opening"
transfer_file_sheet = "transfers"
picked_file_sheet = "picked"
placed_file_sheet = "placed"

In [72]:
converter_dic = {'BookCode' : str, 'Number': int, 'FromAlmirah': str, 
                 'FromRack': int, 'ToAlmirah': str, 'ToRack':int, 
                 'Comments' : str}

opening = pd.read_excel(transfer_file, sheet_name=opening_file_sheet,                        
                         converters=converter_dic, 
                         parse_dates = ['PickListDate'])
transfer = pd.read_excel(transfer_file, sheet_name=transfer_file_sheet, 
                         converters=converter_dic, 
                         parse_dates = ['PickListDate'])
pickup = pd.read_excel(transfer_file, sheet_name=picked_file_sheet, 
                        converters=converter_dic, 
                        parse_dates = ['PickListDate'])
deposit = pd.read_excel(transfer_file, sheet_name=placed_file_sheet, 
                        converters=converter_dic, 
                        parse_dates = ['PickListDate'])



In [73]:
n_opening = opening[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_opening.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_opening.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [74]:
n_tpickup = transfer[['FromAlmirah', 'FromRack', 'BookCode', 'Number']].copy(deep=True)
n_tpickup['PickedNumber'] = - n_tpickup['Number']
n_tpickup.drop('Number', axis=1, inplace=True)
n_tpickup.rename(columns = {'FromAlmirah':'Almirah', 'FromRack':'Rack', 
                            'PickedNumber' : 'Number'}, inplace = True)
n_tpickup.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [75]:
n_tdeposit = transfer[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_tdeposit.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_tdeposit.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [76]:
n_pure_deposit = deposit[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_pure_deposit.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_pure_deposit.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [77]:
n_pure_pickup = pickup[['FromAlmirah', 'FromRack', 'BookCode', 'Number']].copy(deep=True)
n_pure_pickup['PickedNumber'] = - n_pure_pickup['Number']
n_pure_pickup.drop('Number', axis=1, inplace=True)
n_pure_pickup.rename(columns = {'FromAlmirah':'Almirah', 'FromRack':'Rack', 
                                'PickedNumber' : 'Number'}, 
                     inplace = True)
n_pure_pickup.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [78]:
updatedloc = pd.concat([n_opening, n_tpickup, n_tdeposit, 
                        n_pure_pickup, n_pure_deposit], ignore_index=True)

In [79]:
# this cell is only for debug purposes
if debug:
  with pd.ExcelWriter(debug_file) as writer:  
    n_opening.to_excel(writer, sheet_name='Opening')
    n_tpickup.to_excel(writer, sheet_name='TransferPickup')
    n_tdeposit.to_excel(writer, sheet_name='TransferDeposit')
    n_pure_deposit.to_excel(writer, sheet_name='PureDeposit')
    n_pure_pickup.to_excel(writer, sheet_name='PurePickup')
    updatedloc.to_excel(writer, sheet_name='UpdatedLoc')

In [80]:
updated_report = pd.pivot_table(updatedloc, values='Number', 
                                index=['Almirah', 'Rack', 'BookCode'], 
                                aggfunc=np.sum)

updated_report.reset_index(inplace=True)
updated_locations_filtered = updated_report[updated_report['Number'] != 0]

# Bookwise sorting
picklist_sheet = updated_locations_filtered.sort_values(by=['BookCode', 'Number', 
                                                            "Almirah", "Rack"])
# almirah wise sorting
almirah_sheet = updated_locations_filtered.sort_values(by=["Almirah", "Rack", 
                                                           'BookCode', 'Number'])

In [81]:
stock_summary = pd.pivot_table(updated_locations_filtered, values='Number', 
                               index=['BookCode'], aggfunc=np.sum)
stock_summary.sort_values(by=['BookCode'], inplace=True)

In [82]:
# Write down final results
with pd.ExcelWriter(stock_file) as writer:  
    almirah_sheet.to_excel(writer, sheet_name='AlmirahRack')
    picklist_sheet.to_excel(writer, sheet_name='PicklistMaker')
    stock_summary.to_excel(writer, sheet_name='StockSummary')

In [83]:
print(f"Result written to {stock_file}")

Result written to /content/drive/MyDrive/stock/2023-06-01-17-17-43-Stock-Status.xlsx


In [84]:
drive.flush_and_unmount()